What is the most popular category people post to based on the following age groups:

- 18-24
- 25-35
- 36-50
- +50

In [0]:
%run "/Users/sarfaraj_ahmed@outlook.com/Clean"

In [0]:
df_pin.createOrReplaceTempView("pin")
df_user.createOrReplaceTempView("user")

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW categories_per_age AS
SELECT 
  p.category,
  CASE
    WHEN u.age BETWEEN 18 AND 24 THEN '18-24'
    WHEN u.age BETWEEN 25 AND 35 THEN '25-35'
    WHEN u.age BETWEEN 36 AND 50 THEN '36-50'
    WHEN u.age > 50 THEN '+50'
  END AS age_group
FROM
  pin p
INNER JOIN
  user u
ON
  p.ind = u.ind;

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW category_count_age_groups AS
SELECT
  age_group,
  category,
  COUNT(*) AS category_count
FROM
  categories_per_age
GROUP BY
  age_group, 
  category;

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW category_count_rank AS
SELECT
  age_group,
  category,
  category_count,
  ROW_NUMBER() OVER (PARTITION BY age_group ORDER BY category_count DESC) AS rank
FROM
  category_count_age_groups;

In [0]:
%sql
SELECT 
  age_group,
  category,
  category_count
FROM
  category_count_rank
WHERE
  rank = 1
ORDER BY
  CASE
    WHEN age_group = '18-24' THEN 1
    WHEN age_group = '25-35' THEN 2
    WHEN age_group = '36-50' THEN 3
    WHEN age_group = '+50' THEN 4
  END;